In [38]:
# 1. extract video_id, topic_id and category_id, tags
# 2. replace topic_id missing value to NA
# 3. replace tags missing value to "NA"


asso_df = df[["video_id", "topic_id", "category_id", "tags"]]


asso_df.loc[asso_df.topic_id.isnull(), 'topic_id'] = "NA"

asso_df

asso_df.to_csv(f"upload_to_azure/US_association_rule.csv", encoding='utf-8', index=False)

In [9]:
import os
from datetime import datetime
#import isodate
import pandas as pd
import numpy as np

In [10]:
# config, select country 
COUNTRY_CODE = 'US'

# read data frame
df = pd.read_csv(f"final_dataset/{COUNTRY_CODE}videos.csv", parse_dates=["publish_time"])

df['trending_date'] = pd.to_datetime(df['trending_date'])
# 2018-06-12T03:11:18.000Z
df['publish_time'] = pd.to_datetime(df['publish_time'])

In [11]:
print(f"How many rows in { COUNTRY_CODE } table? { len(df) }")
print(f"How many unique videos in { COUNTRY_CODE } table? { len(df.video_id.unique() )}")

How many rows in US table? 47133
How many unique videos in US table? 7323


In [14]:
# first row of the table as a sample
df.tail(3)

,video_id,duration,topic_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,channel_id
47130,zy0b9e40tK8,NaN,NaN,2017-11-14,Dark | Official Trailer [HD] | Netflix,Netflix,24,2017-11-09 09:00:07+00:00,Netflix|Baran Bo Odar|Jantje Friese|DARK|darkm...,378750,5642,146,675,https://i.ytimg.com/vi/zy0b9e40tK8/default.jpg,False,False,False,The disappearance of two kids in the German sm...,NaN
47131,zzQsGL_F9_c,NaN,NaN,2018-02-05,Budweiser | Beer Country | :60,Budweiser,24,2018-01-30 18:30:04+00:00,Budweiser|Super Bowl|Clydesdales|Bud|This Buds...,154206,1180,107,55,https://i.ytimg.com/vi/zzQsGL_F9_c/default.jpg,False,False,False,This is a story bigger than beer. It’s a story...,NaN
47132,zzQsGL_F9_c,NaN,NaN,2018-02-06,Budweiser | Beer Country | :60,Budweiser,24,2018-01-30 18:30:04+00:00,Budweiser|Super Bowl|Clydesdales|Bud|This Buds...,199041,1415,121,62,https://i.ytimg.com/vi/zzQsGL_F9_c/default.jpg,False,False,False,This is a story bigger than beer. It’s a story...,NaN


In [270]:
# the name of the columns and what type of data the columns contain
df.dtypes

video_id                               object
duration                              float64
topic_id                               object
trending_date                  datetime64[ns]
title                                  object
channel_title                          object
category_id                             int64
publish_time              datetime64[ns, UTC]
tags                                   object
views                                   int64
likes                                   int64
dislikes                                int64
comment_count                           int64
thumbnail_link                         object
comments_disabled                        bool
ratings_disabled                         bool
video_error_or_removed                 object
description                            object
channel_id                             object
dtype: object

In [271]:
# VIDEO DURATION (in seconds)

privatized_videos = df[df.duration.isnull()]
print(f"How many privatized / deleted videos entries (NOT UNIQUE) in {COUNTRY_CODE} dataset?", len(privatized_videos))
print(f"How many privatized / deleted videos (UNIQUE) in {COUNTRY_CODE} dataset?", len(privatized_videos.video_id.unique()))

df[["duration"]].describe()

How many privatized / deleted videos entries (NOT UNIQUE) in US dataset? 3968
How many privatized / deleted videos (UNIQUE) in US dataset? 363


,duration
count,43165.000000
mean,630.235306
std,8235.531462
min,0.000000
25%,184.000000
50%,303.000000
75%,619.000000
max,842151.000000


In [272]:
# TOPIC_ID
# the missing value part is the same to video duration
# they both only exist in 2019 new data
# and YouTube doesn't allow us to query privatized/deleted videos

print(f"How many NA values in topic_id column?", len(df[df.topic_id.isnull()]))
print(f"How many videos missing topic_id?", len(df[df.topic_id.isnull()].video_id.unique()))

How many NA values in topic_id column? 4415
How many videos missing topic_id? 427


In [273]:
# topic reference list
topic_id_reference = pd.read_csv("reference/topic_ids.csv", names=["topic_id", "topic_text"], header=None)
topic_id_reference

,topic_id,topic_text
0,/m/04rlf,Music
1,/m/05fw6t,Children's music
2,/m/02mscn,Christian music
3,/m/0ggq0m,Classical music
4,/m/01lyv,Country
5,/m/02lkt,Electronic music
6,/m/0glt670,Hip hop music
7,/m/05rwpb,Independent music
8,/m/03_d0,Jazz
9,/m/028sqc,Music of Asia


In [274]:
# TRENDING_DATE
old_df = df[df.trending_date < pd.Timestamp('2019-10-01')].sort_values('trending_date')
new_df = df[df.trending_date > pd.Timestamp('2019-10-01')].sort_values('trending_date')

old_start = old_df.head(1).trending_date.to_string(index=False)
old_end = old_df.tail(1).trending_date.to_string(index=False)

new_start = new_df.head(1).trending_date.to_string(index=False)
new_end = new_df.tail(1).trending_date.to_string(index=False)

print(f"Kaggle dataset from: { old_start } to { old_end } ")
print(f"New dataset from: { new_start } to { new_end } ")

Kaggle dataset from: 2017-11-14 to 2018-06-14 
New dataset from: 2019-10-15 to 2019-11-04 


In [275]:
# TITLE

print(f"Missing value in title column: { len(df[df.title.isnull()]) }")

Missing value in title column: 0


In [276]:
# CHANNEL_TITLE

print(f"Missing value in channel_title column: { len(df[df.channel_title.isnull()]) }")

Missing value in channel_title column: 0


In [277]:
# CATEGORY_ID

print(f"Missing value in category_id column: { len(df[df.category_id.isnull()]) }")

Missing value in category_id column: 0


In [278]:
# category_id reference

pd.read_csv("reference/category_ids.csv")

,category_id,category_name,category_detail
0,1,Film & Animation,"Film category includes new film trailers, vide..."
1,2,Autos & Vehicles,Autos & Vehicles category includes video relat...
2,10,Music,Music category seems not required any instruct...
3,15,Pets & Animals,Pets and Animals category includes funny and a...
4,17,Sports,Sports category includes videos for sports and...
5,19,Travel & Events,Travel & Events category includes videos for t...
6,20,Gaming,"Gaming category includes videos for games, gam..."
7,22,People & Blogs,"Category includes videos for people, life styl..."
8,23,Comedy,Comedy category includes comedy videos in diff...
9,24,Entertainment,Entertainment category have videos for enterta...


In [279]:
# PUBLISH_TIME
old_df = df[df.publish_time < pd.Timestamp('2019-10-01', tz='UTC')].sort_values('publish_time')
new_df = df[df.publish_time > pd.Timestamp('2019-10-01', tz='UTC')].sort_values('publish_time')

old_start = old_df.head(1).trending_date.to_string(index=False)
old_end = old_df.tail(1).trending_date.to_string(index=False)

new_start = new_df.head(1).trending_date.to_string(index=False)
new_end = new_df.tail(1).trending_date.to_string(index=False)

print(f"Kaggle dataset publish_time ranging from: { old_start } to { old_end } ")
print(f"New dataset publish_time ranging from: { new_start } to { new_end } ")

Kaggle dataset publish_time ranging from: 2018-02-05 to 2018-06-14 
New dataset publish_time ranging from: 2019-10-15 to 2019-11-04 


In [280]:
# TAGS

missing_tags = df[df.tags.isnull()]
print(f'How many rows have missing tags? { len(missing_tags) }')
print(f"How many videos miss tags? { len(missing_tags.video_id.unique()) }")

tags_collector = []
for row in df[df.tags.notnull()].tags:
    tags_collector.extend(row.split('|'))

print(f"How many unique tags in total? { len(set(tags_collector)) }")

How many rows have missing tags? 1775
How many videos miss tags? 296
How many unique tags in total? 65468


In [5]:
# VIEWS, LIKES, DISLIKES, COMMENT_COUNT
df[["views", "likes", "dislikes", "comment_count"]].describe()

,views,likes,dislikes,comment_count
count,4.713300e+04,4.713300e+04,4.713300e+04,4.713300e+04
mean,2.300286e+06,7.449503e+04,3.637262e+03,8.218892e+03
std,7.009090e+06,2.243532e+05,2.754915e+04,3.538383e+04
min,5.490000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.632330e+05,5.665000e+03,2.130000e+02,6.410000e+02
50%,7.170340e+05,1.916100e+04,6.490000e+02,1.957000e+03
75%,1.830129e+06,5.778800e+04,1.937000e+03,5.730000e+03
max,2.252119e+08,5.613827e+06,1.674420e+06,1.361580e+06


In [8]:
df[df.likes == 0]

,video_id,duration,topic_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,channel_id
560,PRHVfIbeGpQ,16.0,/m/019_rr,2017-12-14,How to Write-In for the Alabama Special Electi...,American Bridge 21st Century,25,2017-12-11 14:54:32+00:00,NaN,22690,0,0,0,https://i.ytimg.com/vi/PRHVfIbeGpQ/default.jpg,True,True,False,Ahead of Tuesday's special election in Alabama...,NaN
734,_akwfRuL4os,256.0,/m/098wr,2017-11-23,Highlights of the EA predatory behavior announ...,chris lee,22,2017-11-21 19:29:03+00:00,NaN,160012,0,0,1722,https://i.ytimg.com/vi/_akwfRuL4os/default.jpg,False,True,False,The State of Hawaii announces action to addres...,NaN
735,_akwfRuL4os,256.0,/m/098wr,2017-11-24,Highlights of the EA predatory behavior announ...,Chris Lee,22,2017-11-21 19:29:03+00:00,NaN,179448,0,0,1847,https://i.ytimg.com/vi/_akwfRuL4os/default.jpg,False,True,False,The State of Hawaii announces action to addres...,NaN
736,_akwfRuL4os,256.0,/m/098wr,2017-11-25,Highlights of the EA predatory behavior announ...,Chris Lee,22,2017-11-21 19:29:03+00:00,NaN,187973,0,0,1919,https://i.ytimg.com/vi/_akwfRuL4os/default.jpg,False,True,False,The State of Hawaii announces action to addres...,NaN
737,_akwfRuL4os,256.0,/m/098wr,2017-11-26,Highlights of the EA predatory behavior announ...,Chris Lee,22,2017-11-21 19:29:03+00:00,NaN,193362,0,0,1958,https://i.ytimg.com/vi/_akwfRuL4os/default.jpg,False,True,False,The State of Hawaii announces action to addres...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46981,v4DFeT9z7yY,NaN,NaN,2018-02-22,Camila Cabello Performs 'Never Be the Same' | ...,Dancing On Ice,24,2018-02-18 19:57:11+00:00,dancing on ice|ice skating|camila cabello,593311,0,0,0,https://i.ytimg.com/vi/v4DFeT9z7yY/default.jpg,True,True,False,Camila Cabello lights up the ice with an amazi...,NaN
46982,v4DFeT9z7yY,NaN,NaN,2018-02-23,Camila Cabello Performs 'Never Be the Same' | ...,Dancing On Ice,24,2018-02-18 19:57:11+00:00,dancing on ice|ice skating|camila cabello,759160,0,0,0,https://i.ytimg.com/vi/v4DFeT9z7yY/default.jpg,True,True,False,Camila Cabello lights up the ice with an amazi...,NaN
46983,v4DFeT9z7yY,NaN,NaN,2018-02-24,Camila Cabello Performs 'Never Be the Same' | ...,Dancing On Ice,24,2018-02-18 19:57:11+00:00,dancing on ice|ice skating|camila cabello,923299,0,0,0,https://i.ytimg.com/vi/v4DFeT9z7yY/default.jpg,True,True,False,Camila Cabello lights up the ice with an amazi...,NaN
46984,v4DFeT9z7yY,NaN,NaN,2018-02-25,Camila Cabello Performs 'Never Be the Same' | ...,Dancing On Ice,24,2018-02-18 19:57:11+00:00,dancing on ice|ice skating|camila cabello,1011961,0,0,0,https://i.ytimg.com/vi/v4DFeT9z7yY/default.jpg,True,True,False,Camila Cabello lights up the ice with an amazi...,NaN


In [282]:
# COMMENTS_DISABLED
df['comments_disabled'].value_counts(normalize=True)

False    0.983027
True     0.016973
Name: comments_disabled, dtype: float64

In [283]:
# RATINGS_DISABLED
df['ratings_disabled'].value_counts(normalize=True)

False    0.994887
True     0.005113
Name: ratings_disabled, dtype: float64

In [284]:
# VIDEO_ERROR_OR_REMOVED
df['video_error_or_removed'].value_counts(normalize=True)

False    0.999371
True     0.000629
Name: video_error_or_removed, dtype: float64

In [285]:
# DESCRIPTION

missing_description = df[df.description.isnull()]
print(f"How many missing value in description column? { len(missing_description) } ")
print(f"How many videos miss description? { len(missing_description.video_id.unique()) } ")

How many missing value in description column? 627 
How many videos miss description? 107 


In [ ]:
###### [ARCHIVED - One time used only] ############
## covnert topic_id text to csv
# with open("C:\\Users\\George\\Desktop\\coding-ground\\Youtube-analysis\\reference\\test.txt", "r") as file:
#     file = file.readlines()

# for line in file:
#     seperated = line.split(" ", 1)
#     seperated[1] = seperated[1].replace("\n", "")
#     joined = ",".join(seperated)
    
#     with open("C:\\Users\\George\\Desktop\\coding-ground\\Youtube-analysis\\reference\\topic_ids.csv", "a") as output:
#         output.write(joined + "\n")

In [146]:
###### [ARCHIVED - One time used only] ############
## format category id

# category_r_raw = pd.read_csv("reference/test.csv", sep="\t")

# category_r_raw = category_r_raw.rename(columns={"ID": "category_id", "CATEGORY NAME": "category_name", "CATEGORY DETAIL": "category_detail"})

# category_r_raw.to_csv("reference/category_ids.csv", index=False)